# Problem Set 03: Adversarial Games


0. [Credit for Contributors (required)](#contributors)

2. [Adversarial Games](#problem2)
    1. [Max-Min (50 points)](#part1)
    2. [Max-Min with Alpha-Beta pruning (30 points)](#part2)

    
**80 points** total for Problem Set 3


## <a name="contributors"></a> Credit for Contributors

List the various students, lecture notes, or online resouces that helped you complete this problem set:

Ex: I worked with Bob on the cat activity planning problem.

<div class="alert alert-info">
Write your answer in the cell below this one.
</div>

L11_16_410-13_F23_Multiagent_Games_and_AlphaGo.pptx

Be sure to run the cell below to import the code needed for this assignment.

In [1]:
from __future__ import division

from utils import *
from tests import *

## <a name="problem2"></a> Problem 1: The Cut-Throat Game of Tic-Tac-Toe

Though the game of tic-tac-toe is very simple, it serves as a good example for MiniMax search and alpha-beta pruning because it is fully solvable. In this problem set, you will implement MiniMax search with and without alpha-beta pruning, and compare performance. You will then simulate a full game to prove that an optimal strategy in tic-tac-toe can only guarantee a tie. 

We have defined for you a `game_state` class, which works for general games, and a `tic_tac_toe_board` class, both defined in the file `utils.py`. You may want to look in this file now to see the specifics of these classes.

A `tic_tac_toe_board` object is defined as follows:

```
tic_tac_toe_board(symbols)
```

The `symbols` argument is a list of 9 elements which may be one of `'x'`, `'o'`, or `' '`, corresponding to the symbols on the board supplied row by row.

You will not work directly with a `tic_tac_toe_board`. Instead, a `game_state` object will track the board state of any two player game object supplied to it. It will also provide information on the current player, and the state of the game in the last move. You can think of a `game_state` object as a graph node. An instance is defined as:

```
game_state(board,parent=None)
```

The `board` arguement is any game board, and the `parent` argument refers to the previous game state. If `None` is supplied as the parent, the game is assumed to have started from this state.

To show the current board, you may use `print game_state` as below.

In [2]:
# Define a game state for tic-tac-toe
example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o',' ',' ']))
# View the state of the board
print(example_state)

x | o | x
--+---+--
x | x | o
--+---+--
o |   |  



By default, a `game_state` is initialized assuming player 1, who plays x's and attempts to maximize score, is playing first. Player 2 plays o's and attempts to minimize the score. 

We attribue a score of +1 to any game state where x (player 1) wins, -1 to any game state where o (player 2) wins, and 0 to any tied or incomplete game state. You may see the score using `game_state.score()`, and check whether a game is complete using `game_state.terminal_check()`.

In [3]:
# Run this cell to see some example games, who has won, and whether the game is complete.
example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o',' ',' ']))
print(example_state)
print('Game complete? {}'.format(example_state.terminal_check()))
print('Score of this game is {}: Game incomplete.\n'.format(example_state.score()))

example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o',' ','x']))
print(example_state)
print('Game complete? {}'.format(example_state.terminal_check()))
print('Score of this game is {}: x wins.\n'.format(example_state.score()))

example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o','o','o']))
print(example_state)
print('Game complete? {}'.format(example_state.terminal_check()))
print('Score of this game is {}: o wins.\n'.format(example_state.score()))

example_state = game_state(tic_tac_toe_board(['x','o','x','x','x','o','o','x','o']))
print(example_state)
print('Game complete? {}'.format(example_state.terminal_check()))
print('Score of this game is {}: Game tie.\n'.format(example_state.score()))

x | o | x
--+---+--
x | x | o
--+---+--
o |   |  

Game complete? False
Score of this game is 0: Game incomplete.

x | o | x
--+---+--
x | x | o
--+---+--
o |   | x

Game complete? True
Score of this game is 1: x wins.

x | o | x
--+---+--
x | x | o
--+---+--
o | o | o

Game complete? True
Score of this game is -1: o wins.

x | o | x
--+---+--
x | x | o
--+---+--
o | x | o

Game complete? True
Score of this game is 0: Game tie.



The player who needs to play next in any given `game_state` is given through the command `game_state.player`. The function `game_state.successors()` creates a list of `game_state` objects that could be reachable within a move of the current player. The parent of the successors is automatically allocated.

In [4]:
# Run this cell to see current players and successor states
example_state = game_state(tic_tac_toe_board(['x','o','x','x',' ','o','o',' ',' ']))
print('Example state:')
print(example_state)
print('Next player is {} and the possible successor states are:\n'.format(example_state.player))
successor_states = example_state.successors()
for s in successor_states:
    print(s)
    
print('-----------------------------------------\n')
example_state = successor_states[0]
print('Example state:')
print(example_state)
print('Next player is {} and the possible successor states are:\n'.format(example_state.player))
successor_states = example_state.successors()
for s in successor_states:
    print(s)

Example state:
x | o | x
--+---+--
x |   | o
--+---+--
o |   |  

Next player is 1 and the possible successor states are:

x | o | x
--+---+--
x | x | o
--+---+--
o |   |  

x | o | x
--+---+--
x |   | o
--+---+--
o | x |  

x | o | x
--+---+--
x |   | o
--+---+--
o |   | x

-----------------------------------------

Example state:
x | o | x
--+---+--
x | x | o
--+---+--
o |   |  

Next player is 2 and the possible successor states are:

x | o | x
--+---+--
x | x | o
--+---+--
o | o |  

x | o | x
--+---+--
x | x | o
--+---+--
o |   | o



### <a name="part1"/> Implement MiniMax Search

We will use MiniMax to produce a game where both players are following their optimal strategy. You should be able to convince yourself that in this case, the outcome of the entire game can be defined at the start. 

You will define the following functions:

```python
def maximize_score(state)
def minimize_score(state)
```

Both functions take a `game_state` object as input. It is assumed that the when `maximize_score` is run, the next player to play is player 1, while `minimize_score` is run when the next player is player 2.  

The output of `maximize_score` will be a tuple `(max_score, states_explored, optimal_play)`, where: 

- `max_score` is the maximum score that can be achieved starting from the provided state if both players follow an optimum strategy. 
- `states_explored` is the total number of game states that have been considered to perform this computation (including the current state). If your search reaches the same state multiple times, count it multiple times.
- `optimal_play` is a list of `game_state` objects following the optimal game `[state, state2, state3, ..., final_state]` from the current state (leftmost element of list) to its conclusion (rightmost element).

Similarly, the output of `minimize_score` will be a tuple `(min_score, states_explored, optimal_play)`, where:

- `min_score` is the minimum score that can be achieved starting from the provided state if both players follow an optimum strategy. 
- `states_explored` and `optimal_play` are defined as above.

**In the case where multiple moves give a tie for the optimal play, the `optimal_play` list should follow the first successor in the list generated by `state.successors()`.**

<div class="alert alert-info">
Implement functions `maximize_score(state)` and `minimize_score(state)` below.
</div>

In [18]:
def maximize_score(state):
    max_score = -float("inf")
    states_explored = 1
    optimal_play = [state,]
    for successor in state.successors():
        if successor.terminal_check():
            states_explored += 1
            successor.v = successor.score()
            if successor.v > max_score:
                toadd = [successor,]
                max_score = successor.v
        else:
            result = minimize_score(successor)
            successor.v = result[0]
            states_explored += result[1]
            if successor.v > max_score:
                toadd = result[2]
                max_score = successor.v
    optimal_play = optimal_play + toadd
    return (max_score, states_explored, optimal_play)

def minimize_score(state):
    min_score = float("inf")
    states_explored = 1
    optimal_play = [state,]
    for successor in state.successors():
        if successor.terminal_check():
            states_explored += 1
            successor.v = successor.score()
            if successor.v < min_score:
                toadd = [successor,]
                min_score = successor.v
        else:
            result = maximize_score(successor)
            successor.v = result[0]
            states_explored += result[1]
            if successor.v < min_score:
                toadd = result[2]
                min_score = successor.v
    optimal_play = optimal_play + toadd
    return (min_score, states_explored, optimal_play)

In [19]:
"""Test your minimax search code here."""
test_minimax(maximize_score,minimize_score)

x | o |  
--+---+--
x |   |  
--+---+--
o |   |  

x | o |  
--+---+--
x | x |  
--+---+--
o |   |  

x | o | o
--+---+--
x | x |  
--+---+--
o |   |  

x | o | o
--+---+--
x | x | x
--+---+--
o |   |  



Test passed!!

### <a name="part2"/> Implement Alpha-Beta Pruning

Now you will implement alpha-beta pruning into your minimax functions, and demonstrate that the same result is achieved with fewer game states needing to be searched. 

You will define the following functions:

```python
def maximize_score_alpha_beta(state,alpha,beta)
def minimize_score_alpha_beta(state,alpha,beta)
```

These functions will look very similar to your implementations of `maximize_score` and `minimize_score` (in fact, a good place to start is by copying and pasting your code from above), except they will take the alpha and beta parameters defined by alpha-beta pruning. The output format should be the same as the corresponding functions without pruning.

<div class="alert alert-info">
Implement functions `maximize_score_alpha_beta(state,alpha,beta)` and `minimize_score_alpha_beta(state,alpha,beta)` below.
</div>

In [28]:
# Write your code for minimax search with alpha-beta pruning here.

def maximize_score_alpha_beta(state,alpha,beta):

    if state.player != 1:
        raise Error
    
    # YOUR CODE HERE
    if state.terminal_check():
        return (state.score(),1,[state,])
    
    states_explored = 1
    optimal_play = [state,]
    max_score = -float("inf")
    
    for successor in state.successors():
        """
        if successor.terminal_check():
            states_explored += 1
            successor.v = successor.score()
            if successor.v > max_score:
                toadd = [successor,]
                max_score = successor.v
                if max_score >= beta:
                    optimal_play = optimal_play + toadd
                    return (max_score, states_explored,optimal_play)
        else:
        """
        result = minimize_score_alpha_beta(successor,alpha,beta)
        successor.v = result[0]
        states_explored += result[1]
        if successor.v > max_score:
            toadd = result[2]
            max_score = successor.v
            alpha = max(alpha,max_score)
            if max_score >= beta:
                optimal_play = optimal_play + toadd
                return (max_score, states_explored,optimal_play)
    optimal_play = optimal_play + toadd
    alpha = max(alpha,max_score)
    return (max_score, states_explored, optimal_play)
    

def minimize_score_alpha_beta(state,alpha,beta):

    if state.player != 2:
        raise Error

    # YOUR CODE HERE
    if state.terminal_check():
        return (state.score(),1,[state,])
    
    states_explored = 1
    optimal_play = [state,]
    min_score = float("inf")
    
    for successor in state.successors():
        """
        if successor.terminal_check():
            states_explored += 1
            successor.v = successor.score()
            if successor.v < min_score:
                toadd = [successor,]
                min_score = successor.v
                if min_score <= alpha:
                    optimal_play = optimal_play + toadd
                    return (min_score, states_explored,optimal_play)
        else:
        """
        result = maximize_score_alpha_beta(successor,alpha,beta)
        successor.v = result[0]
        states_explored += result[1]
        if successor.v < min_score:
            toadd = result[2]
            min_score = successor.v
            beta = min(beta,min_score)
            if min_score <= alpha:
                optimal_play = optimal_play + toadd
                return (min_score, states_explored,optimal_play)
    optimal_play = optimal_play + toadd
    beta = min(beta,min_score)
    return (min_score, states_explored, optimal_play)
    

In [29]:
"""Test your alpha beta search code here."""
test_alpha_beta(maximize_score_alpha_beta,minimize_score_alpha_beta)

Test passed!!

### Simulate a Full Game

You're now prepared to run a full game of tic-tac-toe from an initially blank state. If it was possible to always guarantee a win against a perfect opponent, you would see x's win the game. Unfortunately, this isn't possible! Let's see this by running your code boht with and without alpha-beta pruning.

In [30]:
# Run this cell to see an optimum game of tic-tac-toe. Note the performance without alpha-beta pruning. This may
# take a few tens of seconds. 
blank_game = game_state(tic_tac_toe_board([' ',' ',' ',' ',' ',' ',' ',' ',' ']))
(max_score, states_explored, optimal_play) = maximize_score(blank_game)
print('Max score = {}'.format(max_score))
print('States explored = {}'.format(states_explored))
for s in optimal_play:
    print(s)

Max score = 0
States explored = 549946
  |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x |   |  
--+---+--
  | o |  
--+---+--
  |   |  

x | x |  
--+---+--
  | o |  
--+---+--
  |   |  

x | x | o
--+---+--
  | o |  
--+---+--
  |   |  

x | x | o
--+---+--
  | o |  
--+---+--
x |   |  

x | x | o
--+---+--
o | o |  
--+---+--
x |   |  

x | x | o
--+---+--
o | o | x
--+---+--
x |   |  

x | x | o
--+---+--
o | o | x
--+---+--
x | o |  

x | x | o
--+---+--
o | o | x
--+---+--
x | o | x



In [31]:
# Run this cell to see an optimum game of tic-tac-toe. Note the performance gain with alpha-beta pruning.  
blank_game = game_state(tic_tac_toe_board([' ',' ',' ',' ',' ',' ',' ',' ',' ']))
(max_score, states_explored, optimal_play) = maximize_score_alpha_beta(blank_game,-2,2)
print('Max score = {}'.format(max_score))
print('States explored = {}'.format(states_explored))
for s in optimal_play:
    print(s)

Max score = 0
States explored = 18297
  |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x |   |  
--+---+--
  |   |  
--+---+--
  |   |  

x |   |  
--+---+--
  | o |  
--+---+--
  |   |  

x | x |  
--+---+--
  | o |  
--+---+--
  |   |  

x | x | o
--+---+--
  | o |  
--+---+--
  |   |  

x | x | o
--+---+--
  | o |  
--+---+--
x |   |  

x | x | o
--+---+--
o | o |  
--+---+--
x |   |  

x | x | o
--+---+--
o | o | x
--+---+--
x |   |  

x | x | o
--+---+--
o | o | x
--+---+--
x | o |  

x | x | o
--+---+--
o | o | x
--+---+--
x | o | x

